Перед Вами база данных, содержащая задокументированные встречи людей с животными. Данные вносятся в базу из дневников и карточек встреч, поэтому для них нет фиксированного формата комментариев и описаний. Данная информация требуется научным сотрудникам разных заповедников для анализа локаций и частотности встреч людей с разными видами животных.

Вам необходимо ответить на приведенные ниже вопросы на основе представленной таблицы.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('meetings.xlsx', converters={'Номер карточки': int, 'Вид': str, 'Дата':str, 'Тип встречи': str, 'Количество': str, 'Географическая привязка': str, 'Комментарий': str}).iloc[:,:7]

df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d %H:%M:%S', errors='coerce') # coerce - ошибки будут заменены на NaT

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C1160 is marked as a date but the serial value 6651508.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C5874 is marked as a date but the serial value 6572312.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C8144 is marked as a date but the serial value 6606627.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [ ]:
df.head()

,Номер карточки,Вид,Дата,Тип встречи,Количество,Географическая привязка,Комментарий
0,1,"Ovis nivicola nivicola\nEschscholtz, 1829 –\nк...",2011-08-25,визуально,1,"подножье Тауншиц, исток р. Левая Щапина",самец
1,2,"Ovis nivicola nivicola\nEschscholtz, 1829 –\nк...",2011-08-25,визуально,4,"подножье Тауншиц, западный склон, исток ручья ...",2 самки + 2 ягнёнка
2,3,"Ovis nivicola nivicola\nEschscholtz, 1829 –\nк...",2011-01-26,визуально,3,"влк. Кизимен, истоки Поперечного, борт 1371 м",самцы
3,4,"Ovis nivicola nivicola\nEschscholtz, 1829 –\nк...",2011-09-10,след,2,с Дуга (Долина смерти),самцы
4,5,"Ovis nivicola nivicola\nEschscholtz, 1829 –\nк...",2011-08-28,след,NaN,ПС Синий Дол - борт Тауншиц,NaN


#1). В каком столбце больше всего пропущенных значений в исходных данных? Укажите название данного столбца.

In [ ]:
df.isna().sum()

Номер карточки                0
Вид                         337
Дата                        373
Тип встречи                 340
Количество                  349
Географическая привязка     343
Комментарий                6977
dtype: int64

**Ответ: Комментарий**

#2).  Какого числа произошло больше всего визуальных встреч? Укажите ответ в формате yyyy-mm-dd.

In [ ]:
df[df['Тип встречи'] == 'визуально']['Дата'].value_counts().idxmax()

Timestamp('1995-06-02 00:00:00')

**Ответ: 1995-06-02**

#3). Какое максимальное суммарное количество особей одного вида было зафиксировано в зимний период за все время? Укажите ответ в виде целого числа.

Примечание: Рассматривайте только строки с точным указанием количества в соответствующем столбце.

In [ ]:
def is_winter(x):
    if pd.isnull(x):
        return 0
    if int(x) in [12, 1, 2]:
        return 1
    return 0

In [ ]:
tmp = df.copy()

tmp['winter'] = tmp['Дата'].dt.month.apply(lambda x: is_winter(x))

tmp = tmp.dropna(subset=['Количество'])
tmp = tmp[tmp['Количество'].str.isnumeric()]
tmp['Количество'] = tmp['Количество'].astype(int)

tmp[tmp.winter == 1].groupby('Вид').agg({'Количество': sum}).sort_values(by='Количество', ascending=False)

,Количество
Вид,
"Rangifer tarandus\nphylarchus Hollister,\n1912 –дикий\nсеверный олень",30106
"Alces Americana\nbuturlini\nChernyavskyet\nZheleznov, 1982 –\nлось Бутурлина",284
"Lepus timidus gichiganus J. Allen, 1903 – гижигинский заяц-беляк",228
"Ovis nivicola nivicola\nEschscholtz, 1829 –\nкамчатский снежный\nбаран",214
"Gulo gulo albus (Kerr,\n1792) –камчатская\nросомаха",72
"Martes zibellina\ncamtschadalica(Birula,\n1919) – камчатский\nсоболь",54
"Lutra lutra lutra\nLinnaeus, 1758 –\nсеверная речная\nвыдра",39
"Canis lupus\nalbusKerr,1792 –\nполярный волк",34
"Mustela erminea\nkaneii (Baird, 1857) -\nвосточносибирский\nгорностай",27


**Ответ: (Rangifer tarandus phylarchus Hollister, 1912 дикий северный олень в количестве) 30106**

# 4). Какое слово, состоящее более чем из двух символов, встречается в комментариях чаще всего? В ответе укажите данное слово.

In [ ]:
pd.Series(' '.join(df.dropna(subset=['Комментарий'])['Комментарий'].astype(str)).lower().split()).value_counts()[:10]

с           788
2           426
самец       423
и           287
самка       252
1           238
рогами      219
3           205
самцы       184
большими    180
dtype: int64

**Ответ: самец**

# 5). Представители некоторых из упомянутых видов могут проникать в дома. Какой вид был замечен в воровстве? В качестве ответа укажите полное название вида (так, как оно записано в исходных данных). Если таких видов несколько, укажите в ответе их названия через “;”

Посмотрим глазами в экселе слова, которые характеризуют воровство, используем их.

In [ ]:
import re

In [ ]:
t = df.copy()
t = t.dropna(subset=['Комментарий'])

In [ ]:
t[t['Комментарий'].str.contains('проник|похит|залез|ограб', flags=re.IGNORECASE, regex=True)][['Вид', 'Комментарий']]

,Вид,Комментарий
874,"Martes zibellina\ncamtschadalica(Birula,\n1919...",залез в дом
897,"Martes zibellina\ncamtschadalica(Birula,\n1919...","ночью залез в дом, ограбил, устроил погром"
1382,"Mustela erminea\nkaneii (Baird, 1857) -\nвосто...",проникновение
1427,"Mustela erminea\nkaneii (Baird, 1857) -\nвосто...","пробежал по столу, залез на шкаф и смотрел на ..."
1443,"Mustela erminea\nkaneii (Baird, 1857) -\nвосто...","пытался проникнуть, грохотал"
1444,"Mustela erminea\nkaneii (Baird, 1857) -\nвосто...",пытался похитить колбасу


Ответ:

* Martes zibellina
camtschadalica(Birula,
1919) – камчатский
соболь

* Mustela erminea kaneii (Baird, 1857) - восточносибирский горностай

# 6). На каком расстоянии в километрах от Москвы (55.7504461, 37.6174943) были замечены северные олени во время кормления, для которых в качестве географической привязки указаны точные координаты? В ответе укажите целое число.

In [ ]:
t[t['Комментарий'].str.contains('кормились', flags=re.IGNORECASE, regex=True) & t['Вид'].str.contains('северный олень', flags=re.IGNORECASE, regex=True)]

,Номер карточки,Вид,Дата,Тип встречи,Количество,Географическая привязка,Комментарий
5567,5568,"Rangifer tarandus\nphylarchus Hollister,\n1912...",2017-03-10,визуально,6,"N 54.26,86 E 159.51,39",кормились на тундре
5568,5569,"Rangifer tarandus\nphylarchus Hollister,\n1912...",2017-03-10,визуально,26,"N 54.27,02 E 159.50,11",кормились на тундре
5569,5570,"Rangifer tarandus\nphylarchus Hollister,\n1912...",2017-03-10,визуально,60,"N 54.27,17 E 159.49,86",кормились на тундре


https://www.eddmaps.org/tools/latlongconv.cfm

N 54.26,86 E 159.51,39 = (54.457222, 159.860833)

N 54.27,02 E 159.50,11 = (54.450556, 159.836389)

N 54.27,17 E 159.49,86 = (54.454722, 159.840556)

In [ ]:
!pip install haversine

In [ ]:
from haversine import haversine, Unit

In [ ]:
p1 = (54.457222, 159.860833)
p2 = (54.450556, 159.836389)
p3 = (54.454722, 159.840556)

In [ ]:
haversine((55.7504461, 37.6174943), p1, unit='km')

6686.121795247739

In [ ]:
haversine((55.7504461, 37.6174943), p2, unit='km')

6685.873794646475

In [ ]:
haversine((55.7504461, 37.6174943), p3, unit='km')

6685.634548355686

In [ ]:
import geopy.distance

print(geopy.distance.geodesic((55.7504461, 37.6174943), p1).km)
print(geopy.distance.geodesic((55.7504461, 37.6174943), p2).km)
print(geopy.distance.geodesic((55.7504461, 37.6174943), p3).km)

6708.522637917058
6708.272534956781
6708.033332597782


**Ответ: возможны разные варианты, один из верных - 6686**

In [ ]:
import pandas as pd

add = pd.read_csv("answer.csv")

In [ ]:
add

,Номер карточки
0,25
1,57
2,58
3,59
4,60
...,...
522,9655
523,9656
524,9657
525,9658


In [ ]:
df_small = df[df['Номер карточки'].isin(add['Номер карточки'].values)]

In [ ]:
df_small['Вид'].value_counts()

Rangifer tarandus\nphylarchus Hollister,\n1912 –дикий\nсеверный олень    527
Name: Вид, dtype: int64

In [ ]:
df_small['Месяц'] = df_small['Дата'].dt.month

<ipython-input-17-4154d76af23d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['Месяц'] = df_small['Дата'].dt.month


In [ ]:
len(df_small), len(df_small[df_small['Месяц'].isin([12,1,2])])

(527, 521)

In [ ]:
df_small[df_small['Месяц'].isin([12,1,2])]['Количество'].astype(int).sum()

29951

In [ ]:
df_small[~df_small['Месяц'].isin([12,1,2])]

,Номер карточки,Вид,Дата,Тип встречи,Количество,Географическая привязка,Комментарий,Месяц
2245,2246,"Rangifer tarandus\nphylarchus Hollister,\n1912...",2010-03-01,визуально,5,Семячик лиман - Синий дол ...,NaN,3
4796,4797,"Rangifer tarandus\nphylarchus Hollister,\n1912...",1996-03-01,визуально,31,Исток - Аэродром на...,2 группы: 25 + 6,3
5187,5188,"Rangifer tarandus\nphylarchus Hollister,\n1912...",1978-03-01,иное,300,"к югу от Тауншица, в сторону р. Новый Семячик,...",место кормёжки,3
5194,5195,"Rangifer tarandus\nphylarchus Hollister,\n1912...",1978-03-01,след,4,между Тауншицем и г. Крайней,NaN,3
9512,9513,"Rangifer tarandus\nphylarchus Hollister,\n1912...",1991-03-01,визуально,100,Кроноцкий аэродром,NaN,3
9545,9546,"Rangifer tarandus\nphylarchus Hollister,\n1912...",1991-03-01,визуально,50,г. Двухличная,домашние олени рассеялись по тундре и смешалис...,3


In [ ]:
add[add['Номер карточки'].isin([2246,5188])]

,Номер карточки
48,2246
194,5188
